In [1]:
import cv2
import time
import os
from concurrent.futures import ThreadPoolExecutor

# Function to process a single video or HLS stream
def process_video_stream(url, output_folder, frame_skip=10, duration=120):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Open the stream
    cap = cv2.VideoCapture(url)
    if not cap.isOpened():
        print(f"Error: Cannot open stream: {url}")
        return

    # Start time
    start_time = time.time()
    frame_count = 0

    while True:
        # Check if the duration has elapsed
        elapsed_time = time.time() - start_time
        if elapsed_time > duration:
            print(f"Finished processing stream: {url}")
            break

        ret, frame = cap.read()
        if not ret:
            print(f"Error reading frame from {url}. Ending stream processing.")
            break

        # Capture frame every `frame_skip` intervals
        if frame_count % frame_skip == 0:
            # Resize the frame to a smaller size if necessary (optional)
            frame = cv2.resize(frame, (640, 480))

            # Save the frame as an image
            filename = os.path.join(output_folder, f"frame_{frame_count}.jpg")
            cv2.imwrite(filename, frame)
            print(f"Saved: {filename}")

        frame_count += 1

    # Release the capture object
    cap.release()
    print(f"Stream processing complete for {url}")

# Function to process multiple streams in parallel
def process_multiple_streams(urls, base_output_folder="../outputs", frame_skip=10, duration=120):
    # Use ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor() as executor:
        futures = []
        for idx, url in enumerate(urls):
            output_folder = os.path.join(base_output_folder, f"frames_video{idx+1}")
            futures.append(
                executor.submit(process_video_stream, url, output_folder, frame_skip, duration)
            )

        # Wait for all threads to complete
        for future in futures:
            future.result()

# URLs for the HLS streams
urls = [
    "https://manifest.googlevideo.com/api/manifest/hls_playlist/expire/1733751621/ei/5Z5WZ_i0Lemj9fwPvv2q6Qk/ip/183.82.97.138/id/B0YjuKbVZ5w.4/itag/96/source/yt_live_broadcast/requiressl/yes/ratebypass/yes/live/1/sgoap/gir%3Dyes%3Bitag%3D140/sgovp/gir%3Dyes%3Bitag%3D137/rqh/1/hdlc/1/hls_chunk_host/rr5---sn-5jucgv5qc5oq-itqy.googlevideo.com/xpc/EgVo2aDSNQ%3D%3D/playlist_duration/30/manifest_duration/30/vprv/1/playlist_type/DVR/initcwndbps/1253750/met/1733730022,/mh/6g/mm/44/mn/sn-5jucgv5qc5oq-itqy/ms/lva/mv/m/mvi/5/pl/19/rms/lva,lva/dover/11/pacing/0/keepalive/yes/fexp/51326932,51331020,51335594,51347746/mt/1733729547/sparams/expire,ei,ip,id,itag,source,requiressl,ratebypass,live,sgoap,sgovp,rqh,hdlc,xpc,playlist_duration,manifest_duration,vprv,playlist_type/sig/AJfQdSswRAIgVUyEJh-XM-luNNDhezTkcgLhUa1CRZWUbt0A9GJwzXgCIBmQDkv29qZgo18RLQnXTZ50VnhlYXO1Puz3UWWlJRPD/lsparams/hls_chunk_host,initcwndbps,met,mh,mm,mn,ms,mv,mvi,pl,rms/lsig/AGluJ3MwRQIhAMkpcgs6T_6riFQo_WKZ6PY36DeRUbjU9_bZ0Ymxs4E8AiB3YckQtdh9baU44Wrc_pfsTJVNfzgDoDNKa3O41vEIZA%3D%3D/playlist/index.m3u8",
    "https://manifest.googlevideo.com/api/manifest/hls_playlist/expire/1733751692/ei/LJ9WZ7GhL82r9fwP8M6b2Ak/ip/183.82.97.138/id/pw0wrJu-dHg.5/itag/96/source/yt_live_broadcast/requiressl/yes/ratebypass/yes/live/1/sgoap/gir%3Dyes%3Bitag%3D140/sgovp/gir%3Dyes%3Bitag%3D137/rqh/1/hdlc/1/hls_chunk_host/rr2---sn-5jucgv5qc5oq-itqy.googlevideo.com/xpc/EgVo2aDSNQ%3D%3D/vprv/1/playlist_type/LIVE/initcwndbps/1302500/met/1733730093,/mh/48/mm/44/mn/sn-5jucgv5qc5oq-itqy/ms/lva/mv/m/mvi/2/pl/19/rms/lva,lva/dover/11/pacing/0/keepalive/yes/fexp/51326932,51331020,51335594,51347747/mt/1733729788/sparams/expire,ei,ip,id,itag,source,requiressl,ratebypass,live,sgoap,sgovp,rqh,hdlc,xpc,vprv,playlist_type/sig/AJfQdSswRQIgFRv1SEi7ug06S-JAUsT7hIsHksqUPGGo2QdlJsxcdcwCIQCHA40Nzn2nKLysJWB3w39Y9Q2Jybwq18HRsuv71XmiLw%3D%3D/lsparams/hls_chunk_host,initcwndbps,met,mh,mm,mn,ms,mv,mvi,pl,rms/lsig/AGluJ3MwRgIhALTrH4skges6gxUFGZALWPj2mo-M0T-PYAD0xkuLja-3AiEA6jkbZNDkL3pcpSbLz1Hds4NoDU_jAdczp6qtjOlgFs8%3D/playlist/index.m3u8",
    "https://manifest.googlevideo.com/api/manifest/hls_playlist/expire/1733755392/ei/oK1WZ6S0HvrIssUPhcGU6A0/ip/183.82.97.138/id/F5Q5ViU8QR0.1/itag/96/source/yt_live_broadcast/requiressl/yes/ratebypass/yes/live/1/sgoap/gir%3Dyes%3Bitag%3D140/sgovp/gir%3Dyes%3Bitag%3D137/rqh/1/hdlc/1/hls_chunk_host/rr1---sn-5jucgv5qc5oq-itqd.googlevideo.com/xpc/EgVo2aDSNQ%3D%3D/vprv/1/playlist_type/LIVE/initcwndbps/1326250/met/1733733793,/mh/Up/mm/44/mn/sn-5jucgv5qc5oq-itqd/ms/lva/mv/m/mvi/1/pcm2cms/yes/pl/19/rms/lva,lva/dover/11/pacing/0/keepalive/yes/fexp/51326932,51331020,51335594,51347746/mt/1733733385/sparams/expire,ei,ip,id,itag,source,requiressl,ratebypass,live,sgoap,sgovp,rqh,hdlc,xpc,vprv,playlist_type/sig/AJfQdSswRgIhAOsIS1rhkuhr3Gb7JWaR9-8YGmEo7VGRoU5JU45kDnFtAiEA_y6JpByKQptEl9nPDRMkEBjZr4LKDG_4yOYDGZD34o8%3D/lsparams/hls_chunk_host,initcwndbps,met,mh,mm,mn,ms,mv,mvi,pcm2cms,pl,rms/lsig/AGluJ3MwRgIhAJH4yCCsrVHs4ZBxH8UQmJBG5h0ByrgdIAMTm8AQXVCoAiEAoz3ZEuvjx1uu-6wt-F5re9eVvJY4mJd2eJTmi85ofzY%3D/playlist/index.m3u8",
    "https://manifest.googlevideo.com/api/manifest/hls_playlist/expire/1733751823/ei/r59WZ7W0JOyI9fwPnczI0Qk/ip/183.82.97.138/id/VR-x3HdhKLQ.28/itag/96/source/yt_live_broadcast/requiressl/yes/ratebypass/yes/live/1/sgoap/gir%3Dyes%3Bitag%3D140/sgovp/gir%3Dyes%3Bitag%3D137/rqh/1/hdlc/1/hls_chunk_host/rr8---sn-5jucgv5qc5oq-itqy.googlevideo.com/xpc/EgVo2aDSNQ%3D%3D/playlist_duration/30/manifest_duration/30/vprv/1/playlist_type/DVR/initcwndbps/1302500/met/1733730224,/mh/v8/mm/44/mn/sn-5jucgv5qc5oq-itqy/ms/lva/mv/m/mvi/8/pl/19/rms/lva,lva/dover/11/pacing/0/keepalive/yes/fexp/51326932,51331020,51335594,51347746/mt/1733729788/sparams/expire,ei,ip,id,itag,source,requiressl,ratebypass,live,sgoap,sgovp,rqh,hdlc,xpc,playlist_duration,manifest_duration,vprv,playlist_type/sig/AJfQdSswRQIgU4r-XJNyXbuEp4JiBGSs-jmEyXCXjy8dMM9DNKkemeQCIQCAObq5XcyzleNewwzS2wyTVo3I_OV04ED4si2L6DcQtQ%3D%3D/lsparams/hls_chunk_host,initcwndbps,met,mh,mm,mn,ms,mv,mvi,pl,rms/lsig/AGluJ3MwRgIhAL3WB9J9f1Gx7bNlIcM4elE6f0b5ILDhjZik_2Xn-92gAiEAwEwQ3R-TR4mN1gLkB0Uoa4rOjN7nLRXJrAO-LLm9quk%3D/playlist/index.m3u8"
]

# Process all streams in parallel
process_multiple_streams(urls, frame_skip=10, duration=120)


Saved: ../outputs\frames_video4\frame_0.jpg
Saved: ../outputs\frames_video4\frame_10.jpg
Saved: ../outputs\frames_video4\frame_20.jpg
Saved: ../outputs\frames_video4\frame_30.jpg
Saved: ../outputs\frames_video4\frame_40.jpg
Saved: ../outputs\frames_video4\frame_50.jpg
Saved: ../outputs\frames_video4\frame_60.jpg
Saved: ../outputs\frames_video3\frame_0.jpg
Saved: ../outputs\frames_video4\frame_70.jpg
Saved: ../outputs\frames_video3\frame_10.jpg
Saved: ../outputs\frames_video4\frame_80.jpg
Saved: ../outputs\frames_video3\frame_20.jpg
Saved: ../outputs\frames_video4\frame_90.jpg
Saved: ../outputs\frames_video4\frame_100.jpg
Saved: ../outputs\frames_video4\frame_110.jpg
Saved: ../outputs\frames_video4\frame_120.jpg
Saved: ../outputs\frames_video4\frame_130.jpg
Saved: ../outputs\frames_video4\frame_140.jpg
Saved: ../outputs\frames_video3\frame_30.jpg
Saved: ../outputs\frames_video3\frame_40.jpg
Saved: ../outputs\frames_video3\frame_50.jpg
Saved: ../outputs\frames_video3\frame_60.jpg
Saved: 

In [2]:
from ultralytics import YOLO

# Load YOLO Model
model = YOLO("models/yolo11n.pt")

# Vehicle Detection Function
def detect_vehicles(image_path):
    results = model(image_path)
    vehicles = len(results[0].boxes)  # Count bounding boxes
    return vehicles

# Test Detection on One Frame
vehicle_count = detect_vehicles("../outputs/frames_video1/frame_100.jpg")
print(f"Detected {vehicle_count} vehicles in frame_100.jpg")


image 1/1 c:\Users\Admin\Desktop\Traffic Management System\ML Model\notebooks\..\outputs\frames_video1\frame_100.jpg: 480x640 (no detections), 347.1ms
Speed: 14.0ms preprocess, 347.1ms inference, 12.0ms postprocess per image at shape (1, 3, 480, 640)
Detected 0 vehicles in frame_100.jpg


In [3]:
import os
import pandas as pd

def generate_metrics_for_folders(frame_folders, output_folder="../outputs/"):
    os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists
    
    for frame_folder in frame_folders:
        folder_name = os.path.basename(frame_folder.rstrip("/"))
        csv_filename = f"{folder_name}_metrics.csv"  # Generate CSV file name
        csv_path = os.path.join(output_folder, csv_filename)
        
        print(f"Processing folder: {frame_folder}")
        metrics = []
        
        for frame in os.listdir(frame_folder):
            frame_path = os.path.join(frame_folder, frame)
            
            # Detect vehicles in the frame (replace with your actual detection logic)
            vehicle_count = detect_vehicles(frame_path)
            metrics.append({"Frame": frame, "Vehicle_Count": vehicle_count})
        
        # Save metrics to CSV
        metrics_df = pd.DataFrame(metrics)
        metrics_df.to_csv(csv_path, index=False)
        
        print(f"Metrics saved to {csv_path}")

# Example Usage
frame_folders = [
    "../outputs/frames_video1/",
    "../outputs/frames_video2/",
    "../outputs/frames_video3/",
    "../outputs/frames_video4/"
]

generate_metrics_for_folders(frame_folders)


Processing folder: ../outputs/frames_video1/

image 1/1 c:\Users\Admin\Desktop\Traffic Management System\ML Model\notebooks\..\outputs\frames_video1\frame_0.jpg: 480x640 (no detections), 569.2ms
Speed: 201.3ms preprocess, 569.2ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 c:\Users\Admin\Desktop\Traffic Management System\ML Model\notebooks\..\outputs\frames_video1\frame_10.jpg: 480x640 (no detections), 216.2ms
Speed: 17.1ms preprocess, 216.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 c:\Users\Admin\Desktop\Traffic Management System\ML Model\notebooks\..\outputs\frames_video1\frame_100.jpg: 480x640 (no detections), 211.8ms
Speed: 3.0ms preprocess, 211.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 c:\Users\Admin\Desktop\Traffic Management System\ML Model\notebooks\..\outputs\frames_video1\frame_1000.jpg: 480x640 (no detections), 207.0ms
Speed: 5.0ms preprocess, 207.0ms inference, 1.0ms po

In [4]:
import os
import pandas as pd
import math
import cv2
from ultralytics import YOLO

# Constants for speed calculation
FRAME_RATE = 10  # frames per second (adjust based on your video properties)
TIME_INTERVAL = 1 / FRAME_RATE  # seconds
SCALING_FACTOR = 0.05  # meters per pixel (adjust based on calibration)

# Load YOLO Model
model = YOLO("models/yolo11n.pt")

# Vehicle Detection Function
def detect_vehicles(image_path):
    results = model(image_path)
    vehicles = []
    for box in results[0].boxes:
        # Calculate center of bounding box
        x_center = (box.xyxy[0][0] + box.xyxy[0][2]) / 2
        y_center = (box.xyxy[0][1] + box.xyxy[0][3]) / 2
        vehicles.append((x_center.item(), y_center.item()))  # Convert to Python floats
    print(f"Frame: {image_path}, Detected Vehicles: {vehicles}")
    return vehicles

# Match positions between consecutive frames
def match_positions(previous_positions, current_positions):
    matched_positions = []
    for prev in previous_positions:
        if not current_positions:
            break
        # Find the closest match in current positions
        closest_match = min(current_positions, key=lambda curr: math.sqrt((curr[0] - prev[0])**2 + (curr[1] - prev[1])**2))
        matched_positions.append(closest_match)
    return matched_positions

# Calculate average speeds for all frames
def calculate_speeds_in_kmph(frames_folder, scaling_factor, time_interval):
    frames = sorted(os.listdir(frames_folder))  # Ensure frames are processed in order
    previous_positions = []  # Track positions of vehicles in previous frame
    speeds = []  # Store average speeds for each frame

    for i, frame in enumerate(frames):
        frame_path = os.path.join(frames_folder, frame)
        current_positions = detect_vehicles(frame_path)

        if i > 0:  # Calculate speed only after the first frame
            matched_positions = match_positions(previous_positions, current_positions)
            frame_speeds = []

            for prev, curr in zip(previous_positions, matched_positions):
                dx = curr[0] - prev[0]
                dy = curr[1] - prev[1]
                pixel_displacement = math.sqrt(dx**2 + dy**2)
                real_displacement = pixel_displacement * scaling_factor  # meters
                speed_mps = real_displacement / time_interval  # meters per second
                speed_kmph = speed_mps * 3.6  # Convert to km/h

                frame_speeds.append(speed_kmph)

            # Calculate average speed for the frame
            avg_speed = sum(frame_speeds) / len(frame_speeds) if frame_speeds else 0
            speeds.append(avg_speed)
        else:
            speeds.append(0)  # No speed for the first frame

        previous_positions = current_positions  # Update for next iteration

    return speeds

# Update CSV file with speeds
def update_csv_with_speeds(csv_path, frames_folder):
    # Load the existing metrics CSV file
    metrics_df = pd.read_csv(csv_path)
    print(f"Loaded metrics from {csv_path}")

    # Calculate speeds
    avg_speeds_kmph = calculate_speeds_in_kmph(frames_folder, SCALING_FACTOR, TIME_INTERVAL)

    # Align lengths: pad or truncate avg_speeds_kmph to match CSV rows
    if len(avg_speeds_kmph) > len(metrics_df):
        print(f"Truncating speed values: {len(avg_speeds_kmph)} to match CSV rows: {len(metrics_df)}")
        avg_speeds_kmph = avg_speeds_kmph[:len(metrics_df)]  # Truncate
    elif len(avg_speeds_kmph) < len(metrics_df):
        print(f"Padding speed values: {len(avg_speeds_kmph)} to match CSV rows: {len(metrics_df)}")
        avg_speeds_kmph.extend([0] * (len(metrics_df) - len(avg_speeds_kmph)))  # Pad with zeros

    # Add a new column for average speeds (km/h)
    metrics_df['Average_Speed_kmph'] = avg_speeds_kmph

    # Save the updated CSV file
    updated_csv_path = csv_path
    metrics_df.to_csv(updated_csv_path, index=False)
    print(f"Updated metrics saved to {updated_csv_path}")

# Main Execution
if __name__ == "__main__":
    # Paths for frame folders and corresponding CSV files
    frame_folders_and_csvs = [
        {"frames_folder": "../outputs/frames_video1/", "metrics_csv": "../outputs/frames_video1_metrics.csv"},
        {"frames_folder": "../outputs/frames_video2/", "metrics_csv": "../outputs/frames_video2_metrics.csv"},
        {"frames_folder": "../outputs/frames_video3/", "metrics_csv": "../outputs/frames_video3_metrics.csv"},
        {"frames_folder": "../outputs/frames_video4/", "metrics_csv": "../outputs/frames_video4_metrics.csv"},
    ]

    # Iterate over each frame folder and update its respective CSV
    for item in frame_folders_and_csvs:
        frames_folder = item["frames_folder"]
        metrics_csv_path = item["metrics_csv"]

        # Ensure the frames folder and CSV file exist
        if not os.path.exists(frames_folder):
            print(f"Error: Frames folder '{frames_folder}' not found.")
            continue
        if not os.path.exists(metrics_csv_path):
            print(f"Error: Metrics CSV file '{metrics_csv_path}' not found.")
            continue

        # Update the CSV with speed calculations
        update_csv_with_speeds(metrics_csv_path, frames_folder)


Loaded metrics from ../outputs/frames_video1_metrics.csv

image 1/1 c:\Users\Admin\Desktop\Traffic Management System\ML Model\notebooks\..\outputs\frames_video1\frame_0.jpg: 480x640 (no detections), 313.6ms
Speed: 13.4ms preprocess, 313.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Frame: ../outputs/frames_video1/frame_0.jpg, Detected Vehicles: []

image 1/1 c:\Users\Admin\Desktop\Traffic Management System\ML Model\notebooks\..\outputs\frames_video1\frame_10.jpg: 480x640 (no detections), 258.0ms
Speed: 10.0ms preprocess, 258.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Frame: ../outputs/frames_video1/frame_10.jpg, Detected Vehicles: []

image 1/1 c:\Users\Admin\Desktop\Traffic Management System\ML Model\notebooks\..\outputs\frames_video1\frame_100.jpg: 480x640 (no detections), 514.5ms
Speed: 19.8ms preprocess, 514.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Frame: ../outputs/frames_video1/frame_100.jpg, Detected Ve

In [5]:
import os
import pandas as pd
import math
from ultralytics import YOLO

# Constants
SCALING_FACTOR = 0.05  # meters per pixel (adjust based on calibration)

# Load YOLO Model
model = YOLO("models/yolo11n.pt")

# Vehicle Detection Function
def detect_vehicles(image_path):
    results = model(image_path)
    vehicles = []
    for box in results[0].boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Bounding box coordinates
        vehicles.append((x1.item(), y1.item(), x2.item(), y2.item()))  # Convert to Python floats
    print(f"Frame: {image_path}, Detected Vehicles: {vehicles}")
    return vehicles

# Calculate Queue Length
def calculate_queue_length(vehicles, axis="y"):
    """
    Calculate queue length based on vehicle bounding boxes.
    
    Args:
        vehicles (list): List of bounding boxes [(x1, y1, x2, y2), ...].
        axis (str): 'x' for horizontal queue or 'y' for vertical queue.
        
    Returns:
        float: Queue length in meters.
    """
    if not vehicles:
        return 0  # No vehicles detected, queue length is 0
    
    if axis == "x":  # Horizontal queue
        queue_positions = [((x1 + x2) / 2) for x1, y1, x2, y2 in vehicles]  # X centers
    elif axis == "y":  # Vertical queue
        queue_positions = [((y1 + y2) / 2) for x1, y1, x2, y2 in vehicles]  # Y centers
    else:
        raise ValueError("Invalid axis. Use 'x' for horizontal or 'y' for vertical.")

    # Sort vehicle positions along the queue axis
    queue_positions.sort()

    # Calculate total pixel distance
    total_pixel_distance = 0
    for i in range(1, len(queue_positions)):
        total_pixel_distance += abs(queue_positions[i] - queue_positions[i - 1])

    # Convert to meters
    queue_length_meters = total_pixel_distance * SCALING_FACTOR
    return queue_length_meters

# Update CSV with Queue Length
def update_csv_with_queue_length(csv_path, frames_folder, axis="y"):
    # Load the existing metrics CSV file
    metrics_df = pd.read_csv(csv_path)
    print(f"Loaded metrics from {csv_path}")

    # Calculate queue lengths for all frames
    queue_lengths = []
    for frame in sorted(os.listdir(frames_folder)):
        frame_path = os.path.join(frames_folder, frame)
        vehicles = detect_vehicles(frame_path)
        queue_length = calculate_queue_length(vehicles, axis=axis)
        queue_lengths.append(queue_length)
        print(f"Frame: {frame}, Queue Length: {queue_length:.2f} meters")

    # Align lengths: truncate or pad the queue_lengths to match the number of rows in the CSV
    if len(queue_lengths) > len(metrics_df):
        print(f"Truncating queue lengths: {len(queue_lengths)} to match CSV rows: {len(metrics_df)}")
        queue_lengths = queue_lengths[:len(metrics_df)]  # Truncate to match the CSV rows
    elif len(queue_lengths) < len(metrics_df):
        print(f"Padding queue lengths: {len(queue_lengths)} to match CSV rows: {len(metrics_df)}")
        queue_lengths.extend([0] * (len(metrics_df) - len(queue_lengths)))  # Pad with zeros

    # Add queue length column to the dataframe
    metrics_df['Queue_Length_meters'] = queue_lengths

    # Save the updated CSV file
    updated_csv_path = csv_path
    metrics_df.to_csv(updated_csv_path, index=False)
    print(f"Updated metrics saved to {updated_csv_path}")

# Main Execution
if __name__ == "__main__":
    # Paths for frame folders and corresponding CSV files
    frame_folders_and_csvs = [
        {"frames_folder": "../outputs/frames_video1/", "metrics_csv": "../outputs/frames_video1_metrics.csv"},
        {"frames_folder": "../outputs/frames_video2/", "metrics_csv": "../outputs/frames_video2_metrics.csv"},
        {"frames_folder": "../outputs/frames_video3/", "metrics_csv": "../outputs/frames_video3_metrics.csv"},
        {"frames_folder": "../outputs/frames_video4/", "metrics_csv": "../outputs/frames_video4_metrics.csv"},
    ]

    # Iterate over each frame folder and update its respective CSV
    for item in frame_folders_and_csvs:
        frames_folder = item["frames_folder"]
        metrics_csv_path = item["metrics_csv"]

        # Ensure the frames folder and CSV file exist
        if not os.path.exists(frames_folder):
            print(f"Error: Frames folder '{frames_folder}' not found.")
            continue
        if not os.path.exists(metrics_csv_path):
            print(f"Error: Metrics CSV file '{metrics_csv_path}' not found.")
            continue

        # Update the CSV with queue length calculations
        # Choose 'x' for horizontal queues or 'y' for vertical queues
        update_csv_with_queue_length(metrics_csv_path, frames_folder, axis="y")


Loaded metrics from ../outputs/frames_video1_metrics.csv

image 1/1 c:\Users\Admin\Desktop\Traffic Management System\ML Model\notebooks\..\outputs\frames_video1\frame_0.jpg: 480x640 (no detections), 651.6ms
Speed: 6.4ms preprocess, 651.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Frame: ../outputs/frames_video1/frame_0.jpg, Detected Vehicles: []
Frame: frame_0.jpg, Queue Length: 0.00 meters

image 1/1 c:\Users\Admin\Desktop\Traffic Management System\ML Model\notebooks\..\outputs\frames_video1\frame_10.jpg: 480x640 (no detections), 248.8ms
Speed: 6.0ms preprocess, 248.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Frame: ../outputs/frames_video1/frame_10.jpg, Detected Vehicles: []
Frame: frame_10.jpg, Queue Length: 0.00 meters

image 1/1 c:\Users\Admin\Desktop\Traffic Management System\ML Model\notebooks\..\outputs\frames_video1\frame_100.jpg: 480x640 (no detections), 567.2ms
Speed: 6.0ms preprocess, 567.2ms inference, 1.0ms postprocess per 

In [6]:
import pandas as pd
import os

# Congestion Classification Function
def classify_congestion(queue_length):
    """
    Classify congestion levels based on queue length.
    
    Args:
        queue_length (float): Queue length in meters.
        
    Returns:
        str: Congestion level ('Low', 'Medium', 'High').
    """
    if queue_length <= 10:
        return "Low"
    elif 10 < queue_length <= 20:
        return "Medium"
    else:
        return "High"

# Update CSV with Congestion Levels
def update_csv_with_congestion_levels(csv_path):
    # Load the existing metrics CSV file
    metrics_df = pd.read_csv(csv_path)
    print(f"Loaded metrics from {csv_path}")

    # Ensure 'Queue_Length_meters' column exists
    if 'Queue_Length_meters' not in metrics_df.columns:
        print("Error: 'Queue_Length_meters' column not found in the CSV file.")
        return

    # Classify congestion levels for each frame
    metrics_df['Congestion_Level'] = metrics_df['Queue_Length_meters'].apply(classify_congestion)

    # Save the updated CSV file
    updated_csv_path = csv_path
    metrics_df.to_csv(updated_csv_path, index=False)
    print(f"Updated metrics with congestion levels saved to {updated_csv_path}")

# Main Execution
if __name__ == "__main__":
    # Paths to the metrics CSV files
    metrics_csv_files = [
        "../outputs/frames_video1_metrics.csv",
        "../outputs/frames_video2_metrics.csv",
        "../outputs/frames_video3_metrics.csv",
        "../outputs/frames_video4_metrics.csv",
    ]

    # Process each CSV file
    for csv_path in metrics_csv_files:
        if not os.path.exists(csv_path):
            print(f"Error: Metrics CSV file '{csv_path}' not found.")
            continue
        update_csv_with_congestion_levels(csv_path)


Loaded metrics from ../outputs/frames_video1_metrics.csv
Updated metrics with congestion levels saved to ../outputs/frames_video1_metrics.csv
Loaded metrics from ../outputs/frames_video2_metrics.csv
Updated metrics with congestion levels saved to ../outputs/frames_video2_metrics.csv
Loaded metrics from ../outputs/frames_video3_metrics.csv
Updated metrics with congestion levels saved to ../outputs/frames_video3_metrics.csv
Loaded metrics from ../outputs/frames_video4_metrics.csv
Updated metrics with congestion levels saved to ../outputs/frames_video4_metrics.csv


In [7]:
import pandas as pd
import os

# Traffic Density Calculation Function
def calculate_traffic_density(vehicle_count, queue_length):
    """
    Calculate traffic density as vehicles per meter.
    
    Args:
        vehicle_count (int): Number of vehicles detected in the frame.
        queue_length (float): Queue length in meters.
        
    Returns:
        float: Traffic density (vehicles per meter).
    """
    if queue_length == 0:  # Avoid division by zero
        return 0
    return vehicle_count / queue_length

# Update CSV with Traffic Density
def update_csv_with_traffic_density(csv_path):
    # Load the existing metrics CSV file
    metrics_df = pd.read_csv(csv_path)
    print(f"Loaded metrics from {csv_path}")

    # Ensure required columns exist
    if 'Vehicle_Count' not in metrics_df.columns or 'Queue_Length_meters' not in metrics_df.columns:
        print("Error: Required columns 'Vehicle_Count' and 'Queue_Length_meters' are missing.")
        return

    # Calculate traffic density for each frame
    metrics_df['Traffic_Density_vehicles_per_meter'] = metrics_df.apply(
        lambda row: calculate_traffic_density(row['Vehicle_Count'], row['Queue_Length_meters']),
        axis=1
    )

    # Save the updated CSV file
    updated_csv_path = csv_path
    metrics_df.to_csv(updated_csv_path, index=False)
    print(f"Updated metrics with traffic density saved to {updated_csv_path}")

# Main Execution
if __name__ == "__main__":
    # Paths to the metrics CSV files
    metrics_csv_files = [
       "../outputs/frames_video1_metrics.csv",
        "../outputs/frames_video2_metrics.csv",
        "../outputs/frames_video3_metrics.csv",
        "../outputs/frames_video4_metrics.csv",
    ]

    # Process each CSV file
    for csv_path in metrics_csv_files:
        if not os.path.exists(csv_path):
            print(f"Error: Metrics CSV file '{csv_path}' not found.")
            continue
        update_csv_with_traffic_density(csv_path)


Loaded metrics from ../outputs/frames_video1_metrics.csv
Updated metrics with traffic density saved to ../outputs/frames_video1_metrics.csv
Loaded metrics from ../outputs/frames_video2_metrics.csv
Updated metrics with traffic density saved to ../outputs/frames_video2_metrics.csv
Loaded metrics from ../outputs/frames_video3_metrics.csv
Updated metrics with traffic density saved to ../outputs/frames_video3_metrics.csv
Loaded metrics from ../outputs/frames_video4_metrics.csv
Updated metrics with traffic density saved to ../outputs/frames_video4_metrics.csv
